In [1]:
import tensorflow as tf
import pandas as pd

from pathlib import Path

2023-11-28 16:42:35.239522: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 16:42:35.323299: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 16:42:37.043174: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Using a simple RNN

In [2]:
path = Path("datasets/ridership/CTA_-_Ridership_-_Daily_Boarding_Totals.csv")
df = pd.read_csv(path, parse_dates=["service_date"])
# 将数据框的列名更改为较短的形式，以提高代码的可读性。
df.columns = ["date", "day_type", "bus", "rail", "total"]
# 根据日期列对数据框进行排序，并将日期列设置为数据框的索引。
df = df.sort_values("date").set_index("date")
# no need for total, it's just bus + rail
df = df.drop("total", axis=1)
# remove duplicated months (2011-10 and 2014-07)
df = df.drop_duplicates()

rail_train = df["rail"]["2016-01":"2018-12"] / 1e6
rail_valid = df["rail"]["2019-01":"2019-05"] / 1e6
rail_test = df["rail"]["2019-06"]

seq_length = 56
tf.random.set_seed(42)
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_train.to_numpy(),
    targets=rail_train[seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_valid.to_numpy(),
    targets=rail_valid[seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

In [3]:
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

In [4]:
def fit_and_evaluate(model, train_set, valid_set, learning_rate, epochs=500):
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=50, restore_best_weights=True)
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
    history = model.fit(train_set, validation_data=valid_set, epochs=epochs, callbacks=[early_stopping_cb])
    valid_loss, valid_mae = model.evaluate(valid_set)
    return valid_mae * 1e6

In [5]:
fit_and_evaluate(model, train_ds, valid_ds, learning_rate=0.02)

Epoch 1/500
33/33 [==============================] - 2s 21ms/step - loss: 0.1349 - mae: 0.3937 - val_loss: 0.0741 - val_mae: 0.3434
Epoch 2/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0590 - mae: 0.2941 - val_loss: 0.0517 - val_mae: 0.2721
Epoch 3/500
33/33 [==============================] - 0s 11ms/step - loss: 0.0279 - mae: 0.1707 - val_loss: 0.0152 - val_mae: 0.1261
Epoch 4/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0147 - mae: 0.1563 - val_loss: 0.0139 - val_mae: 0.1377
Epoch 5/500
33/33 [==============================] - 0s 11ms/step - loss: 0.0142 - mae: 0.1498 - val_loss: 0.0139 - val_mae: 0.1348
Epoch 6/500
33/33 [==============================] - 0s 12ms/step - loss: 0.0140 - mae: 0.1500 - val_loss: 0.0138 - val_mae: 0.1332
Epoch 7/500
33/33 [==============================] - 1s 14ms/step - loss: 0.0139 - mae: 0.1481 - val_loss: 0.0136 - val_mae: 0.1328
Epoch 8/500
33/33 [==============================] - 0s 12ms/step - loss: 0.

102811.00869178772

### Deep RNNs

In [6]:
tf.random.set_seed(42)  # extra code – ensures reproducibility
univar_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 1]),
    tf.keras.layers.Dense(1)  # no activation function by default
])

In [7]:
# extra code – compiles, fits, and evaluates the model, like earlier
fit_and_evaluate(univar_model, train_ds, valid_ds, learning_rate=0.05)  # return 29473

Epoch 1/500
33/33 [==============================] - 2s 20ms/step - loss: 0.0144 - mae: 0.1306 - val_loss: 0.0057 - val_mae: 0.0859
Epoch 2/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0045 - mae: 0.0645 - val_loss: 0.0025 - val_mae: 0.0484
Epoch 3/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0034 - mae: 0.0524 - val_loss: 0.0021 - val_mae: 0.0378
Epoch 4/500
33/33 [==============================] - 0s 14ms/step - loss: 0.0034 - mae: 0.0527 - val_loss: 0.0023 - val_mae: 0.0419
Epoch 5/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0031 - mae: 0.0500 - val_loss: 0.0019 - val_mae: 0.0345
Epoch 6/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0032 - mae: 0.0542 - val_loss: 0.0020 - val_mae: 0.0361
Epoch 7/500
33/33 [==============================] - 1s 14ms/step - loss: 0.0034 - mae: 0.0539 - val_loss: 0.0022 - val_mae: 0.0372
Epoch 8/500
33/33 [==============================] - 1s 17ms/step - loss: 0.

31252.723187208176

In [8]:
tf.random.set_seed(42)
deep_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[None, 1]),
    tf.keras.layers.SimpleRNN(32, return_sequences=True),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(1)
])

In [9]:
# extra code – compiles, fits, and evaluates the model, like earlier
fit_and_evaluate(deep_model, train_ds, valid_ds, learning_rate=0.01)  # return 30165

Epoch 1/500
33/33 [==============================] - 4s 49ms/step - loss: 0.0336 - mae: 0.1599 - val_loss: 0.0044 - val_mae: 0.0792
Epoch 2/500
33/33 [==============================] - 1s 37ms/step - loss: 0.0067 - mae: 0.0884 - val_loss: 0.0032 - val_mae: 0.0625
Epoch 3/500
33/33 [==============================] - 1s 36ms/step - loss: 0.0063 - mae: 0.0842 - val_loss: 0.0027 - val_mae: 0.0522
Epoch 4/500
33/33 [==============================] - 1s 36ms/step - loss: 0.0047 - mae: 0.0675 - val_loss: 0.0038 - val_mae: 0.0665
Epoch 5/500
33/33 [==============================] - 1s 37ms/step - loss: 0.0042 - mae: 0.0603 - val_loss: 0.0044 - val_mae: 0.0736
Epoch 6/500
33/33 [==============================] - 1s 36ms/step - loss: 0.0043 - mae: 0.0622 - val_loss: 0.0024 - val_mae: 0.0446
Epoch 7/500
33/33 [==============================] - 1s 35ms/step - loss: 0.0041 - mae: 0.0600 - val_loss: 0.0032 - val_mae: 0.0607
Epoch 8/500
33/33 [==============================] - 1s 36ms/step - loss: 0.

29806.658625602722

### Multivariate time series

In [28]:
df_mulvar = df[["bus", "rail"]] / 1e6  # use both bus & rail series as input
df_mulvar["next_day_type"] = df["day_type"].shift(-1)  # we know tomorrow's type
df_mulvar = pd.get_dummies(df_mulvar)  # one-hot encode the day type

df_mulvar["next_day_type_A"] = df_mulvar["next_day_type_A"].astype(int)
df_mulvar["next_day_type_U"] = df_mulvar["next_day_type_U"].astype(int)
df_mulvar["next_day_type_W"] = df_mulvar["next_day_type_W"].astype(int)

In [29]:
mulvar_train = df_mulvar["2016-01":"2018-12"]
mulvar_valid = df_mulvar["2019-01":"2019-05"]
mulvar_test = df_mulvar["2019-06":]

In [30]:
tf.random.set_seed(42)

train_mulvar_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_train.to_numpy(),  # use all 5 columns as input
    targets=mulvar_train["rail"][seq_length:],  # forecast only the rail series
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)

valid_mulvar_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_valid.to_numpy(),
    targets=mulvar_valid["rail"][seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

In [31]:
tf.random.set_seed(42)
mulvar_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 5]),
    tf.keras.layers.Dense(1)
])

In [32]:
fit_and_evaluate(mulvar_model, train_mulvar_ds, valid_mulvar_ds, learning_rate=0.05)

Epoch 1/500
33/33 [==============================] - 2s 19ms/step - loss: 0.0360 - mae: 0.1767 - val_loss: 0.0016 - val_mae: 0.0413
Epoch 2/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0033 - mae: 0.0630 - val_loss: 0.0012 - val_mae: 0.0353
Epoch 3/500
33/33 [==============================] - 0s 12ms/step - loss: 0.0016 - mae: 0.0388 - val_loss: 0.0011 - val_mae: 0.0366
Epoch 4/500
33/33 [==============================] - 0s 12ms/step - loss: 0.0016 - mae: 0.0410 - val_loss: 7.5276e-04 - val_mae: 0.0292
Epoch 5/500
33/33 [==============================] - 0s 12ms/step - loss: 0.0015 - mae: 0.0387 - val_loss: 0.0013 - val_mae: 0.0425
Epoch 6/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0014 - mae: 0.0374 - val_loss: 8.9067e-04 - val_mae: 0.0331
Epoch 7/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0014 - mae: 0.0366 - val_loss: 7.1662e-04 - val_mae: 0.0269
Epoch 8/500
33/33 [==============================] - 0s 13ms/ste

22907.99468755722

In [33]:
# extra code – build and train a multitask RNN that forecasts both bus and rail

tf.random.set_seed(42)

seq_length = 56
train_multask_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_train.to_numpy(),
    targets=mulvar_train[["bus", "rail"]][seq_length:],  # 2 targets per day
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_multask_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_valid.to_numpy(),
    targets=mulvar_valid[["bus", "rail"]][seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

tf.random.set_seed(42)
multask_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 5]),
    tf.keras.layers.Dense(2)
])

fit_and_evaluate(multask_model, train_multask_ds, valid_multask_ds,
                 learning_rate=0.02)

Epoch 1/500
33/33 [==============================] - 2s 23ms/step - loss: 0.0744 - mae: 0.2664 - val_loss: 0.0078 - val_mae: 0.1089
Epoch 2/500
33/33 [==============================] - 1s 15ms/step - loss: 0.0041 - mae: 0.0657 - val_loss: 0.0019 - val_mae: 0.0494
Epoch 3/500
33/33 [==============================] - 1s 14ms/step - loss: 0.0025 - mae: 0.0481 - val_loss: 0.0011 - val_mae: 0.0348
Epoch 4/500
33/33 [==============================] - 1s 14ms/step - loss: 0.0022 - mae: 0.0454 - val_loss: 0.0019 - val_mae: 0.0508
Epoch 5/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0021 - mae: 0.0445 - val_loss: 0.0012 - val_mae: 0.0372
Epoch 6/500
33/33 [==============================] - 0s 13ms/step - loss: 0.0020 - mae: 0.0427 - val_loss: 0.0010 - val_mae: 0.0341
Epoch 7/500
33/33 [==============================] - 1s 15ms/step - loss: 0.0019 - mae: 0.0418 - val_loss: 0.0010 - val_mae: 0.0351
Epoch 8/500
33/33 [==============================] - 0s 13ms/step - loss: 0.

26092.7751660347

26092

In [36]:
# evaluates the naive forecasts for bus
bus_naive = mulvar_valid["bus"].shift(7)[seq_length:]
bus_target = mulvar_valid["bus"][seq_length:]
(bus_target - bus_naive).abs().mean() * 1e6

43441.63157894738

In [37]:
# evaluates the multitask RNN's forecasts both bus and rail
Y_preds_valid = multask_model.predict(valid_multask_ds)
for idx, name in enumerate(["bus", "rail"]):
    mae = 1e6 * tf.keras.metrics.mean_absolute_error(
        mulvar_valid[name][seq_length:], Y_preds_valid[:, idx])
    print(name, int(mae))

3/3 [==============================] - 0s 6ms/step
bus 26630
rail 25554


### Forecasting Several Steps Ahead

In [ ]:
# class LNSimpleRNNCell(tf.keras.layers.Layer):
#     def __init__(self, units, activation="tanh", **kwargs):
#         super().__init__(**kwargs)
#         self.state_size = units
#         self.output_size = units
#         self.simple_rnn_cell = tf.keras.layers.SimpleRNNCell(units, activation=None)
#         self.layer_norm = tf.keras.layers.LayerNormalization()
#         self.activation = tf.keras.activations.get(activation)
    
#     def call(self, inputs, states):
#         outputs, new_states = self.simple_rnn_cell(inputs, states)
#         norm_outputs = self.activation(self.layer_norm(outputs))
#         return norm_outputs, [norm_outputs]

In [ ]:
# tf.random.set_seed(42)
# custom_ln_model = tf.keras.Sequential([
#     tf.keras.layers.RNN(LNSimpleRNNCell(32), return_sequences=True, input_shape=[None, 5]),
#     tf.keras.layers.Dense(14)
# ])

In [ ]:
# fit_and_evaluate(custom_ln_model, seq2seq_train, seq2seq_valid, learning_rate=0.1, epochs=5)